In [2]:
import numpy as np
import pandas as pd
import cantera as ct
import os
import shutil
import optunity

num_pts = 2000 # the number of points to generate
skip = 500     # the number of initial points of the Sobol sequence to skip
# generate Sobol sequence
x_sobol=np.zeros((num_pts, 40))
res = optunity.solvers.Sobol.i4_sobol_generate(40, num_pts, skip)

for i,j in enumerate(zip(*res)):
    x_sobol[:,i]=j
    
percent=0.1

delta=percent*(1-2x)

cti_file='mechanisms/File_0/test111.yaml'
new_file='mechanisms/File_0/test111mod.yaml'

shutil.copyfile(cti_file, new_file)

input_file = open(new_file,'r')
lines = input_file.readlines()
input_file.close()
new_file_content=""

switch=False
for index,line in enumerate(lines):
    if line.strip().startswith('OC(111):'):
        switch=True
        line_idx=index
    elif index == line_idx+2:
        if switch==True:     
            initial=line.strip()
            old=initial.split('[')
            bits=old[1].replace("]","").split(',')
            oldA=float(bits[0])
            oldB=float(bits[1])
            newA=oldA+(oldA*percent-2*x_sobol[0,12]*oldA*percent)
            newB=oldB+(oldB*percent-2*x_sobol[0,13]*oldB*percent)
            modified = old[0] + '[' + str(newA) + ',' + str(newB)+ ',0,0]'
            new_line=line.replace(initial,modified)
            new_file_content += new_line
            print(new_line)
            switch=False
        else:
            initial=line.strip()
            new_line=line.replace(initial,initial)
            new_file_content += new_line
    else:
        new_file_content += line

writing_file = open(new_file, "w")
writing_file.write(new_file_content)
writing_file.close()
    

SyntaxError: invalid decimal literal (817109477.py, line 19)

In [4]:
import shutil
import pandas as pd
import numpy as np
import cantera as ct
import os
import optunity

# Function to modify lines based on specific patterns
def modify_lines(lines, patterns, delta_A, delta_B):
    new_file_content = ""
    switch = False

    for index, line in enumerate(lines):
        if any(line.strip().startswith(pattern) for pattern in patterns):
            switch = True
            new_file_content += line
            line_idx = index
        elif switch and index == line_idx + 2:
            initial = line.strip()
            old = initial.split('[')
            bits=old[1].replace("]","").split(',')
            oldA=float(bits[0])
            oldB=float(bits[1])   
            newA=oldA+(oldA*percent*(1-2*delta_A))
            newB=oldB+(oldB*percent*(1-2*delta_B))         
            modified = old[0] + '[' + str(newA) + ',' + str(newB)+ ',0,0]'
            new_line=line.replace(initial,modified)
            new_file_content += new_line
            switch = False  # Reset switch after modification
        else:
            new_file_content += line

    return new_file_content


num_pts = 10 # the number of points to generate
start=0
skip = 500     # the number of initial points of the Sobol sequence to skip
# generate Sobol sequence
x_sobol=np.zeros((num_pts, 40))
res = optunity.solvers.Sobol.i4_sobol_generate(40, num_pts, skip)

for i,j in enumerate(zip(*res)):
    x_sobol[:,i]=j
    
percent=0.1

directory='mechanisms/File_'

facets=['111','100','211','110']

for i in facets:
    cti_filename = 'test'+i+'.yaml'
    new_filename = 'test'+i+'mod.yaml'

    for k in range(num_pts):
        # Copy the file
        #Create a new folder for the new input file and write the input file
        mkdir_writer="".join((directory,str(k+start)))
        new_file=mkdir_writer + '/' + new_filename
        cti_file=mkdir_writer + '/' + cti_filename
    
        shutil.copyfile(cti_file, new_file)
    
        # Define the patterns and corresponding Ea adjustments
        patterns_list = [
            (['O(111)'], x_sobol[k,12],x_sobol[k,13]),
            (['OC(111)'], x_sobol[k,14],x_sobol[k,15]),
            (['O(100)'], x_sobol[k,16],x_sobol[k,17]),
            (['OC(100)'], x_sobol[k,18],x_sobol[k,19]),
            (['O(110)'], x_sobol[k,20],x_sobol[k,21]),
            (['OC(110)'], x_sobol[k,22],x_sobol[k,23]),
            (['O(211)'], x_sobol[k,24],x_sobol[k,25]),
            (['OC(211)'], x_sobol[k,26],x_sobol[k,27]),
            ]
    
        # Open the file and read the lines
        with open(new_file, 'r') as input_file:
            lines = input_file.readlines()

        # Apply modifications for each pattern set
        for patterns, delta_A, delta_B in patterns_list:
            lines = modify_lines(lines, patterns, delta_A, delta_B).splitlines(keepends=True)

        # Write the final modified content back to the file
        with open(new_file, "w") as writing_file:
            writing_file.write("".join(lines))